In [1]:
# -*- coding: utf-8 -*-
"""
        @Author: Patty Jula
        @Date created: 3/2022
        @Date updated: 6/21/2023
        @Description: This script takes a data wrangled shapefile, converts to
        a json and overwrites an existing feature layer on ArcGIS Online. It
        is useful for content that needs regular updates
"""

'\n        @Author: Patty Jula\n        @Date created: 3/2022\n        @Date updated: 6/21/2023\n        @Description: This script takes a data wrangled shapefile, converts to\n        a json and overwrites an existing feature layer on ArcGIS Online. It\n        is useful for content that needs regular updates\n'

In [5]:
# import libraries
import os
import sys

import arcgis
import arcpy
from arcgis.gis import GIS

from base import get_logger

arcpy.env.overwriteOutput = True

# Set up logger file
logger = get_logger("z_arl_data_v4", "a")
logger.info("Preparing JSON for AGOL")

# Set variables
ws = "E:/batch/arl/content/"
loc = ws
pth = os.chdir(loc)
output_feature_class = "ARL_data.shp"
localJSON = ws + "./arl_content.json"


2023-06-23 11:13:44,471  z_arl_data_v4   INFO  Preparing JSON for AGOL
2023-06-23 11:13:44,471  z_arl_data_v4   INFO  Preparing JSON for AGOL


In [6]:
try:
    if os.path.isfile(localJSON):
        os.remove(localJSON)

except FileNotFoundError:
    print("Wrong file or file path")

# Truncate the existing feature layer

def TruncateWebLayer(gis, target):

    try:
        lyr = arcgis.features.FeatureLayer(target, gis)
        lyr.manager.truncate()
        print("Successfully truncated layer: " + str(target))
    except Exception:
        e = sys.exc_info()[1]
        print(e.args[0])

In [7]:
# LAPD log in, create a profile so you don't need to show un/pw
mygis = GIS("https://lapd.maps.arcgis.com/", profile="python_playground_prof")

# publishedWebLayer is the URL of a single feature layer within a collection in
# an AGOL portal
publishedWebLayer = "https://services6.arcgis.com/Q18o8KwHjFGbEc4j/arcgis/rest/services/ARL_data/FeatureServer/0"

# a feature class on the local system with the same schema as the
# portal layer
updateFeatures = os.path.join("E:/batch/arl/content/" + output_feature_class)

# remove all features from the already published feature layer
feature_layer_item = mygis.content.search("arl whole file")[0]
flayers = feature_layer_item.layers

flayer = flayers[0]

flayer.delete_features(where="area > 0 OR area =0")

{'deleteResults': [{'objectId': 26027677,
   'uniqueId': 26027677,
   'globalId': None,
   'success': True},
  {'objectId': 26027678,
   'uniqueId': 26027678,
   'globalId': None,
   'success': True},
  {'objectId': 26027679,
   'uniqueId': 26027679,
   'globalId': None,
   'success': True},
  {'objectId': 26027680,
   'uniqueId': 26027680,
   'globalId': None,
   'success': True},
  {'objectId': 26027681,
   'uniqueId': 26027681,
   'globalId': None,
   'success': True},
  {'objectId': 26027682,
   'uniqueId': 26027682,
   'globalId': None,
   'success': True},
  {'objectId': 26027683,
   'uniqueId': 26027683,
   'globalId': None,
   'success': True},
  {'objectId': 26027684,
   'uniqueId': 26027684,
   'globalId': None,
   'success': True},
  {'objectId': 26027685,
   'uniqueId': 26027685,
   'globalId': None,
   'success': True},
  {'objectId': 26027686,
   'uniqueId': 26027686,
   'globalId': None,
   'success': True},
  {'objectId': 26027687,
   'uniqueId': 26027687,
   'globalId'

In [8]:
# reference the empty layer as FeatureLayer object from the ArcGIS Python API
fl = arcgis.features.FeatureLayer(publishedWebLayer, mygis)

# create a JSON object from the local features
jSON = arcpy.FeaturesToJSON_conversion(updateFeatures, localJSON)

# create a FeatureSet object from the JSON

fs = arcgis.features.FeatureSet.from_json(open(localJSON).read())
# logstuff("Creating a featureset from the JSON")

# add/append the local features to the hosted feature layer

fl.edit_features(adds=fs)
# logstuff("Appending latest data to the hosted feature layer")
logger.info(arcpy.GetMessages(1))

logger.info("Finishing steps for load to AGOL")


2023-06-23 11:14:52,707  z_arl_data_v4   INFO  
2023-06-23 11:14:52,707  z_arl_data_v4   INFO  
2023-06-23 11:14:52,709  z_arl_data_v4   INFO  Finishing steps for load to AGOL
2023-06-23 11:14:52,709  z_arl_data_v4   INFO  Finishing steps for load to AGOL
